In [ ]:
###### Import Libraries #########

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Covid19-Dash'

app.layout = html.Div(children=[
    # All elements from the top of the page
    html.Div([

        html.Div([html.H1("COVID-19 Dashboard")],
                                style={'textAlign': "center", "padding-bottom": "30"}
                               ),    

        html.Div([html.Span("Metric to display: ", className="six columns",
                        style={"text-align": "right", "width": "30%", "padding-top": '10'}
                           ),
              dcc.Dropdown(id='value-selected', value='Total Confirmed Cases',
                           options=[{'label': "Total Confirmed Cases", 'value': 'Total Confirmed Cases'},
                                    {'label': "Total Deaths", 'value': 'Total Deaths'}],
                            style={"display": "block", "margin-left": "auto", "margin-right": "auto",
                                                     "width": "70%"},
                            className="six columns")], className="row"),
    
        html.Div([
            html.H1(children=' '),


            html.Div(children='''
                COVID-19 Impact on Countries
            '''),

            dcc.Graph(
                id='graph1'
            ),  
        ], style={'textAlign': "center", "padding-bottom": "30"} , className='five columns'),

            
        html.Div([
            html.H1(children=' '),
        
            html.Div(children='''
                COVID-19 Impact on the United States
            '''),

            dcc.Graph(
                id='graph2'
            ),  
        ], style={'textAlign': "center", "padding-bottom": "30"} , className='five columns'),

        
    ], className='row')

])

@app.callback(
    Output('graph1', 'figure'),
    Output('graph2', 'figure'),
    Input('value-selected', 'value'))


def fig_generator(selected):
   
    # Get COVID-19 data from data.world source
    url1 = 'https://query.data.world/s/pbjaa2s4fndnowj4bzc2lfw7cu5nvw'
    df = pd.read_csv(url1, low_memory=False, parse_dates=['REPORT_DATE'])

   # Group by Country
    df_country = df.groupby(['COUNTRY_SHORT_NAME']).sum().reset_index()

    # Get Country Code
    code_df = df[['COUNTRY_SHORT_NAME', 'COUNTRY_ALPHA_3_CODE']]
    code_df = code_df.drop_duplicates(keep='first')    
    # Merge two dataset
    df_country_code = df_country.merge(code_df, left_on='COUNTRY_SHORT_NAME', right_on='COUNTRY_SHORT_NAME', how='left')
    df_country_code = df_country_code[['COUNTRY_SHORT_NAME', 'COUNTRY_ALPHA_3_CODE', 'PEOPLE_POSITIVE_NEW_CASES_COUNT', 'PEOPLE_DEATH_NEW_COUNT']]
    df_country_code = df_country_code.rename(columns={'COUNTRY_SHORT_NAME': 'Country_Region',
                               'COUNTRY_ALPHA_3_CODE': 'CODE',
                               'PEOPLE_POSITIVE_NEW_CASES_COUNT' : 'Total Confirmed Cases',
                               'PEOPLE_DEATH_NEW_COUNT': 'Total Deaths'})
    
    countries_fig = px.choropleth(df_country_code, locations="CODE",
                    color=selected,
                    hover_name="Country_Region",
                    color_continuous_scale=px.colors.sequential.Reds)

    fig_layout = go.Layout(title = " This fig is generated using plotly  ")
    
    fig1 = go.Figure( data = countries_fig ,layout = fig_layout)
    
    
    df_us = df[df['COUNTRY_SHORT_NAME'] == 'United States']

    st_grp = df_us.groupby(['PROVINCE_STATE_NAME']).sum().reset_index()
    
    state_code = pd.read_excel("states code.xlsx")
    
    df_state_code = st_grp.merge(state_code, left_on='PROVINCE_STATE_NAME', right_on='State', how='left')
    
    df_state_code = df_state_code[['State', 'Code', 'PEOPLE_POSITIVE_NEW_CASES_COUNT', 'PEOPLE_DEATH_NEW_COUNT']]

    df_state_code = df_state_code.rename(columns={'PEOPLE_POSITIVE_NEW_CASES_COUNT' : 'Total Confirmed Cases',
                               'PEOPLE_DEATH_NEW_COUNT': 'Total Deaths'})
    
    states_fig = px.choropleth(df_state_code, locations='Code', 
                           locationmode="USA-states", 
                           scope="usa",
                           color=selected,
                           hover_name="State",
                           color_continuous_scale=px.colors.sequential.Reds)
    
    fig_layout = go.Layout(title = " This fig is generated using plotly  ")
    
    fig2 = go.Figure( data = states_fig ,layout = fig_layout)

    return fig1, fig2

   
if __name__=='__main__':
    app.run_server()